In [1]:
pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


In [2]:
#Import dependencies

import pandas as pd
from sqlalchemy import create_engine

In [3]:
#Import ratings data csv
ratings_csv = './ratings.csv'

ratings_df = pd.read_csv(ratings_csv)
ratings_df.head()


,game_title,platform,other_platforms,publisher,developer,release_date,description,genre,ESRB_Rating,multiplayer,...,num_meta_reviews,userscore,num_user_reviews,critic_positive,critic_mixed,critic_negative,user_positive,user_mixed,user_negative,URL
0,Farming Simulator 22,PlayStation_5,"['PC', 'PlayStation 4', 'Xbox_One', 'Xbox_Seri...","GiantsSoftware\n\n\t\t\t\t\t,\t\t\t\t\t\t\nSol...",Giants Software,22-Nov-21,Assume the role of a modern farmer in three di...,Simulation,E,NaN,...,5,5.9,7.0,4,1,0,3,0,1,https://www.metacritic.com/game/playstation-5/...
1,Sherlock Holmes: Chapter One,PlayStation_5,"['PC', 'Xbox_Series_X']",Frogwares,Frogwares,16-Nov-21,"In this story-driven detective thriller, a yo...",Action Adventure,M,NaN,...,26,7.5,46.0,15,11,0,12,2,1,https://www.metacritic.com/game/playstation-5/...
2,Battlefield 2042,PlayStation_5,"['PC', 'PlayStation 4', 'Xbox_One', 'Xbox_Seri...",ElectronicArts,EA DICE,12-Nov-21,Battlefield 2042 marks the return to the iconi...,Action,M,64,...,11,3.0,840.0,3,8,0,94,34,313,https://www.metacritic.com/game/playstation-5/...
3,Grand Theft Auto: The Trilogy - The Definitive...,PlayStation_5,"['PC', 'PlayStation 4', 'Switch', 'Xbox_One', ...",RockstarGames,"Rockstar Games, Grove Street Games",11-Nov-21,"Three iconic cities, three epic stories. Play ...",Miscellaneous,NaN,NaN,...,38,0.9,4045.0,5,26,7,101,42,"1,434",https://www.metacritic.com/game/playstation-5/...
4,The Elder Scrolls V: Skyrim Anniversary Edition,PlayStation_5,"['PC', 'PlayStation 4', 'Xbox_One', 'Xbox_Seri...",BethesdaSoftworks,Bethesda Game Studios,11-Nov-21,The Anniversary Edition includes Skyrim Specia...,Role-Playing,M,NaN,...,10,3.2,59.0,7,3,0,3,2,16,https://www.metacritic.com/game/playstation-5/...


In [4]:
#Import sales data csv
sales_csv = './sales.csv'

sales_df = pd.read_csv(sales_csv)
sales_df.head()

,Rank,Name,Platform,Year,Genre,Publisher,NorthAmerica_Sales,EurpeanUnion_Sales,Japan_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008,Racing,Nintendo,NaN,12.88,3.79,NaN,35.82
3,4,Wii Sports Resort,Wii,2009,Sports,Nintendo,15.75,11.01,NaN,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [5]:
#Select columns from ratings_df for analysis
ratings_sliced=ratings_df[['game_title','platform','ESRB_Rating','metascore','userscore']]

#Rename some columns to clarify what the data is
ratings_renamed=ratings_sliced.rename(columns={'ESRB_Rating':'esrb_rating',
                                          'userscore':'user_score'})
ratings_renamed.head()

,game_title,platform,esrb_rating,metascore,user_score
0,Farming Simulator 22,PlayStation_5,E,77,5.9
1,Sherlock Holmes: Chapter One,PlayStation_5,M,75,7.5
2,Battlefield 2042,PlayStation_5,M,68,3.0
3,Grand Theft Auto: The Trilogy - The Definitive...,PlayStation_5,NaN,56,0.9
4,The Elder Scrolls V: Skyrim Anniversary Edition,PlayStation_5,M,74,3.2


In [6]:
#Check to see if any data needs to be dropped
ratings_renamed.count()

game_title     17101
platform       17101
esrb_rating    13750
metascore      17101
user_score     14776
dtype: int64

In [7]:
#Drop NaN values to clean up data
ratings_final = ratings_renamed.dropna()

print(ratings_final.count())
ratings_final.head()

game_title     12144
platform       12144
esrb_rating    12144
metascore      12144
user_score     12144
dtype: int64


,game_title,platform,esrb_rating,metascore,user_score
0,Farming Simulator 22,PlayStation_5,E,77,5.9
1,Sherlock Holmes: Chapter One,PlayStation_5,M,75,7.5
2,Battlefield 2042,PlayStation_5,M,68,3.0
4,The Elder Scrolls V: Skyrim Anniversary Edition,PlayStation_5,M,74,3.2
6,Call of Duty: Vanguard,PlayStation_5,M,74,3.9


In [8]:
#Select columns from sales_df for analysis
sales_sliced=sales_df[['Name','Platform','Global_Sales']]

sales_renamed=sales_sliced.rename(columns={'Name':'game_title',
                                           'Platform':'platform',
                                          'Global_Sales':'global_sales'})

sales_renamed.head()

,game_title,platform,global_sales
0,Wii Sports,Wii,82.74
1,Super Mario Bros.,NES,40.24
2,Mario Kart Wii,Wii,35.82
3,Wii Sports Resort,Wii,33.00
4,Pokemon Red/Pokemon Blue,GB,31.37


In [9]:
#Check to see if any data needs to be dropped
sales_renamed.count()

game_title      199
platform        199
global_sales    187
dtype: int64

In [10]:
#Drop NaN values to clean up data
sales_final = sales_renamed.dropna()

print(sales_final.count())
sales_final.head()

game_title      187
platform        187
global_sales    187
dtype: int64


,game_title,platform,global_sales
0,Wii Sports,Wii,82.74
1,Super Mario Bros.,NES,40.24
2,Mario Kart Wii,Wii,35.82
3,Wii Sports Resort,Wii,33.00
4,Pokemon Red/Pokemon Blue,GB,31.37


In [11]:
#Merge the two final dataframes
merged_df=pd.merge(ratings_final,sales_final,on=['game_title','atform'])
merged_df.head()

,game_title,platform,esrb_rating,metascore,user_score,global_sales
0,Grand Theft Auto V,Xbox_One,M,97,7.8,5.08
1,Grand Theft Auto V,Xbox_One,M,97,7.8,5.08
2,Call of Duty: Advanced Warfare,Xbox_One,M,81,5.5,5.13
3,Mario & Sonic at the Olympic Games,Wii,E,67,7.7,8.06
4,Mario Kart 64,Nintendo_64,E,83,8.6,9.87


In [12]:
#Drop duplicates to clean the merged dataset
merged_cleaned_df = merged_df.drop_duplicates()
merged_cleaned_df.head()

,game_title,platform,esrb_rating,metascore,user_score,global_sales
0,Grand Theft Auto V,Xbox_One,M,97,7.8,5.08
2,Call of Duty: Advanced Warfare,Xbox_One,M,81,5.5,5.13
3,Mario & Sonic at the Olympic Games,Wii,E,67,7.7,8.06
4,Mario Kart 64,Nintendo_64,E,83,8.6,9.87
5,Mario Kart 8,Wii_U,E,88,8.8,6.96


In [13]:
#Import config file with password, username and database name from Postgres
from config import password,username,database


In [14]:
#Connect to database
rds_connection_string = f"{username}:{password}@localhost:5432/{database}"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [15]:
#Check for tables
engine.table_names()

<ipython-input-15-3a4280413f14>:2: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['ratings', 'sales']

In [17]:
#Insert ratings data into Database
ratings_final.to_sql(name='ratings', con=engine, if_exists='append', index=False)

In [19]:
#Insert sales data into Database
sales_final.to_sql(name='sales', con=engine, if_exists='append', index=False)